In [52]:
from numpy import zeros, transpose, asarray, sum,  diag, dot, arccos
from numpy.linalg import norm
import numpy
from scipy.linalg import svd, inv
import matplotlib.pyplot as plt
import re, random, pylab
from math import * 
from operator import itemgetter

In [53]:
# stopwords, retreived from http://www.lextek.com/manuals/onix/stopwords1.html

stopwords = ['a', 'about', 'above', 'across', 'after', 'again', 'against', 
'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 
'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 
'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 
'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 
'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 
'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 
'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 
'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 
'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 
'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 
'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 
'felt', 'few', 'find', 'finds', 'first', 'for', 'four', 'from', 'full', 'fully', 
'further', 'furthered', 'furthering', 'furthers', 'g', 'gave', 'general', 'generally', 
'get', 'gets', 'give', 'given', 'gives', 'go', 'going', 'good', 'goods', 'got', 'great', 
'greater', 'greatest', 'group', 'grouped', 'grouping', 'groups', 'h', 'had', 'has', 'have', 
'having', 'he', 'her', 'here', 'herself', 'high', 'high', 'high', 'higher', 'highest', 
'him', 'himself', 'his', 'how', 'however', 'i', 'if', 'important', 'in', 'interest', 
'interested', 'interesting', 'interests', 'into', 'is', 'it', 'its', 'itself', 'j', 
'just', 'k', 'keep', 'keeps', 'kind', 'knew', 'know', 'known', 'knows', 'l', 'large', 'largely', 
'last', 'later', 'latest', 'least', 'less', 'let', 'lets', 'like', 'likely', 'long', 'longer',
'longest', 'm', 'made', 'make', 'making', 'man', 'many', 'may', 'me', 'member', 'members', 
'men', 'might', 'more', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 
'n', 'necessary', 'need', 'needed', 'needing', 'needs', 'never', 'new', 'new', 'newer',
'newest', 'next', 'no', 'nobody', 'non', 'noone', 'not', 'nothing', 'now', 'nowhere', 
'number', 'numbers', 'o', 'of', 'off', 'often', 'old', 'older', 'oldest', 'on',
'once', 'one', 'only', 'open', 'opened', 'opening', 'opens', 'or', 'order', 
'ordered', 'ordering', 'orders', 'other', 'others', 'our', 'out', 'over', 'p', 
'part', 'parted', 'parting', 'parts', 'per', 'perhaps', 'place', 'places', 'point',
'pointed', 'pointing', 'points', 'possible', 'present', 'presented', 'presenting', 
'presents', 'problem', 'problems', 'put', 'puts', 'q', 'quite', 'r', 'rather', 
'really', 'right', 'right', 'room', 'rooms', 's', 'said', 'same', 'saw', 'say', 
'says', 'second', 'seconds', 'see', 'seem', 'seemed', 'seeming', 'seems', 
'sees', 'several', 'shall', 'she', 'should', 'show', 'showed', 'showing', 
'shows', 'side', 'sides', 'since', 'small', 'smaller', 'smallest', 'so', 
'some', 'somebody', 'someone', 'something', 'somewhere', 'state', 'states', 
'still', 'still', 'such', 'sure', 't', 'take', 'taken', 'than', 'that', 'the', 
'their', 'them', 'then', 'there', 'therefore', 'these', 'they', 'thing', 'things', 
'think', 'thinks', 'this', 'those', 'though', 'thought', 'thoughts', 'three', 
'through', 'thus', 'to', 'today', 'together', 'too', 'took', 'toward', 'turn', 
'turned', 'turning', 'turns', 'two', 'u', 'under', 'until', 'up', 'upon', 
'us', 'use', 'used', 'uses', 'v', 'very', 'w', 'want', 'wanted', 'wanting', 
'wants', 'was', 'way', 'ways', 'we', 'well', 'wells', 'went', 'were', 
'what', 'when', 'where', 'whether', 'which', 'while', 'who', 'whole', 'whose', 
'why', 'will', 'with', 'within', 'without', 'work', 'worked', 'working', 'works', 
'would', 'x', 'y', 'year', 'years', 'yet', 'you', 'young', 'younger', 'youngest', 'your', 
'yours', 'z']
ignore_characters = ''',:'!'''

In [54]:
def compare(query1, query2): # core comparison function.
    query1=query1 + " Dummy"
    query2=query2 + " Dummy"
    lsa = LSA(stopwords, ignore_characters)
    queries = [query1,query2]
    for q in queries:
        lsa.parse(q)
    lsa.build()
    lsa.calc()
    Vt = lsa.Vt
    S = diag(lsa.S)
    #print(Vt.shape)
    #print(S.shape)
    vectors =[(dot(S,Vt[:,0]),dot(S,Vt[:,i])) for i in range(len(Vt))]
    angles = [arccos(dot(a,b)/(norm(a,2)*norm(b,2))) for a,b in vectors[1:]]
    #print(len(vectors))
    #print(len(angles))
    if len(angles) == 0:
        return 0
    else :
        return str(abs(1 - float(angles[0])/float(pi/2)))

In [55]:
class LSA(object):
    def __init__(self, stopwords, ignore_characters):
        self.stopwords = stopwords
        self.ignore_characters = ignore_characters
        self.wdict = {}
        self.dcount = 0        
    def parse(self, doc):
        words = doc.split();
        for w in words:
            w = w.lower()
            #.translate(None, self.ignore_characters)
            if w in self.stopwords:
                continue
            elif w in self.wdict:
                self.wdict[w].append(self.dcount)
            else:
                self.wdict[w] = [self.dcount]
        self.dcount += 1      
    def build(self): # Create count matrix
        self.keys = [k for k in self.wdict.keys() if len(self.wdict[k]) > 1]
        #if len(self.keys)==0:
         #   self.wdict["Dummy"] = [0]
          #  self.keys.append("Dummy")
        self.keys.sort()
        self.A = zeros([len(self.keys), self.dcount])
        for i, k in enumerate(self.keys):
            for d in self.wdict[k]:
                self.A[i,d] += 1
    def calc(self): # execute SVD
        #print(self.A.shape)
        self.U, self.S, self.Vt = svd(self.A, full_matrices =False)
    def TFIDF(self): # calculate tfidf score
        WordsPerDoc = sum(self.A, axis=0)        
        DocsPerWord = sum(asarray(self.A > 0, 'i'), axis=1)
        rows, cols = self.A.shape
        for i in range(rows):
            for j in range(cols):
                self.A[i,j] = (self.A[i,j] / WordsPerDoc[j]) * log(float(cols) / DocsPerWord[i])
    def S(self):
        return self.S
    def U(self):
        return -1 * self.U
    def Vt(self):
        return -1 * self.Vt
    #def printSVD(self):
        #print('Singular values: ')
        #print(self.S)
        #print('U matrix: ')
        #print(-1*self.U[:, 0:3])
        #print('Vt matrix:) '
        #print(-1*self.Vt[0:3, :])

In [56]:
compare('30-year-old Moscow resident was hospitalized with wounds very intimate nature. As it became known LifeNews, in the hands of doctors, the man complained that his casual acquaintance opoila in the sauna, and then gently held his castration operation. And actions criminals were executed with surgical precision - woman sewed all the smallest blood vessels.', 'Mark Zuckerberg')

0

In [57]:
compare('Hundreds of Palestinians were evacuated from their homes Sunday morning after Israeli authorities opened a number of dams near the border, flooding the Gaza Valley in the wake of a recent severe winter storm.','Hundreds of Palestinians flee floods in Gaza as Israel opens dams')

'1.0'

In [58]:
compare('Many are expecting Apple talk more about the Apple Watch. However, sources familiar with the matter within Apple have exclusively told The Michael Report that Apple plans to unveil the long-awaited Retina MacBook Air at the same event. The Michael Report has independently verified this information to be highly credible. We should note that this information falls in line with what Apple analysts such as KGI Securities’ Ming-Chi Kuo and Oppenheimer’s Andrew Uerkwitz have been predicting for months. Supply chain sources who spoke to the press have also said that Apple ramped up their production in late 2014 and entered the mass production stage in December, with the goal of producing enough units for an early-2015 debut. Last week, Apple sent out the invites for its “Spring Forward” event, slated to be held at the Yerba Buena Center for the Arts in San Francisco on March 9th.','Woman has surgery to get third breast: The three most statistically unlikely pairs of boobs')

'0.07627295930784783'

In [59]:
type(compare('Many are expecting Apple talk more about the Apple Watch. However, sources familiar with the matter within Apple have exclusively told The Michael Report that Apple plans to unveil the long-awaited Retina MacBook Air at the same event. The Michael Report has independently verified this information to be highly credible. We should note that this information falls in line with what Apple analysts such as KGI Securities’ Ming-Chi Kuo and Oppenheimer’s Andrew Uerkwitz have been predicting for months. Supply chain sources who spoke to the press have also said that Apple ramped up their production in late 2014 and entered the mass production stage in December, with the goal of producing enough units for an early-2015 debut. Last week, Apple sent out the invites for its “Spring Forward” event, slated to be held at the Yerba Buena Center for the Arts in San Francisco on March 9th.','Woman has surgery to get third breast: The three most statistically unlikely pairs of boobs'))

str

In [60]:
from csv import DictReader


class DataSet():
    def __init__(self, name="train", path="fnc-1"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [91]:
trainD = DataSet(name="train")

Reading dataset
Total stances: 49972
Total bodies: 1683


In [92]:
trainD.stances[0]

OrderedDict([('Headline',
              "Police find mass graves with at least '15 bodies' near Mexico town where 43 students disappeared after police clash"),
             ('Body ID', 712),
             ('Stance', 'unrelated')])

In [96]:
trainD.stances[0]
for stance in trainD.stances:
    tempStance = stance['Stance']
    if tempStance == 'disagree':
        stance['Stance'] = 'unrelated'
    elif tempStance != 'unrelated':
        stance['Stance'] = 'related'

In [112]:
#Threshold manually varied and re-run several times to find the optimal threshold of 0.18
%%time
threshold=0.18
headlines_train = []
bodies_train = []
y_out_train = []
train_pred = []
Train_feature_list = []
TP=0
TN=0
FP=0
FN=0
for stance in trainD.stances:
    bid = stance['Body ID']
    bodies_train.append(trainD.articles[bid])
    headlines_train.append(stance['Headline'])
    y_out_train.append(stance['Stance']=='related')
    comparison=compare(stance['Headline'],trainD.articles[bid])
    Train_feature_list.append(comparison)
    if float(comparison)>threshold:
        train_pred.append(1)
        #print("Threshold = ",threshold,' Iteration = ',i)
        i+=1
    else:
        train_pred.append(0)
        #print('Iteration = ',i)
        i+=1
correct = [ i==j for i,j in zip(train_pred,y_out_train)]
for i,j in zip(train_pred,y_out_train):
    if (i==1) & (j==1):
        TP+=1
    elif (i==0) & (j==0):
        TN+=1
    elif (i==1) & (j==1):
        FP+=1
    else:
        FN+=1
    
#TP += [(i==1)&(j==1) for i,j in zip(test_pred,y_out_test)] 
#TN += [i==0&j==0 for i,j in zip(test_pred,y_out_test)]
#FP += [i==1&j==0 for i,j in zip(test_pred,y_out_test)]
#FN += [i==0&j==1 for i,j in zip(test_pred,y_out_test)]


print(sum(correct)*1.0/len(correct))
print("done")

0.902705515088
done
Wall time: 2min 8s


In [113]:
print("|TP ",TP,"\n|TN ",TN,"\n|FP ",FP,"\n|FN ",FN)

|TP  9957 
|TN  35153 
|FP  0 
|FN  4862


In [114]:
BER = 1-(0.5*((TP/(TP+FN))+(TN/(TN+FP))))
print("The BER is ", BER)

The BER is  0.1640461569606586


In [116]:
len(Train_feature_list)

49972

In [117]:
print("Pickling body vectors")
import pickle
with open('LSA_Compare_Features_Train.pkl','wb') as f:
    pickle.dump(Train_feature_list,f)
print("Done")

Pickling body vectors
Done


In [61]:
testD = DataSet(name="competition_test")

Reading dataset
Total stances: 25413
Total bodies: 904


In [62]:
testD.stances[0]

OrderedDict([('Headline',
              'Ferguson riots: Pregnant woman loses eye after cops fire BEAN BAG round through car window'),
             ('Body ID', 2008),
             ('Stance', 'unrelated')])

In [63]:
testD.stances[0]
for stance in testD.stances:
    tempStance = stance['Stance']
    if tempStance == 'disagree':
        stance['Stance'] = 'unrelated'
    elif tempStance != 'unrelated':
        stance['Stance'] = 'related'

In [80]:
%%time
threshold=0.18
headlines_test = []
bodies_test = []
y_out_test = []
test_pred = []
feature_list = []
TP=0
TN=0
FP=0
FN=0
for stance in testD.stances:
    bid = stance['Body ID']
    bodies_test.append(testD.articles[bid])
    headlines_test.append(stance['Headline'])
    y_out_test.append(stance['Stance']=='related')
    comparison=compare(stance['Headline'],testD.articles[bid])
    feature_list.append(comparison)
    if float(comparison)>threshold:
        test_pred.append(1)
        #print("Threshold = ",threshold,' Iteration = ',i)
        i+=1
    else:
        test_pred.append(0)
        #print('Iteration = ',i)
        i+=1
correct = [ i==j for i,j in zip(test_pred,y_out_test)]
for i,j in zip(test_pred,y_out_test):
    if (i==1) & (j==1):
        TP+=1
    elif (i==0) & (j==0):
        TN+=1
    elif (i==1) & (j==1):
        FP+=1
    else:
        FN+=1
    
#TP += [(i==1)&(j==1) for i,j in zip(test_pred,y_out_test)] 
#TN += [i==0&j==0 for i,j in zip(test_pred,y_out_test)]
#FP += [i==1&j==0 for i,j in zip(test_pred,y_out_test)]
#FN += [i==0&j==1 for i,j in zip(test_pred,y_out_test)]

t.append(threshold)
a.append(sum(correct)*1.0/len(correct))
print("done")

done
Wall time: 1min 22s


In [89]:
print("|TP ",TP,"\n|TN ",TN,"\n|FP ",FP,"\n|FN ",FN)

|TP  4970 
|TN  17314 
|FP  0 
|FN  3129


In [84]:
BER = 1-(0.5*((TP/(TP+FN))+(TN/(TN+FP))))
print("The BER is ", BER)

The BER is  0.19317199654278305


In [67]:
correct = [ i==j for i,j in zip(test_pred,y_out_test)]
sum(correct)*1.0/len(correct)

0.87687404084523668

In [69]:
feature_list[:16]

['0.043450835427771395',
 '0.07367486408976276',
 '0.07856172352184254',
 '0.029189645979856627',
 '0.140048696093102',
 '0.12566591637800228',
 0,
 '0.06135369737204377',
 '0.09792402541527445',
 '0.1789123750220668',
 '0.4627400919763902',
 0,
 '0.24970466007663183',
 '0.04499710839762605',
 '0.36266749342827453',
 '0.2025148987437202']

In [120]:
print("Pickling body vectors")
import pickle
with open('LSA_Compare_Features_Test.pkl','wb') as f:
    pickle.dump(feature_list,f)
print("Done")

Pickling body vectors
Done


In [119]:
len(feature_list)

25413